## pFound


Нужно написать функцию, которая берет на вход dataframe,а выдает average pFound по всем query.

Notes:

- не использовать циклы для расчет метрики
- попробуйте обойтись без groupby 

In [5]:
import warnings
import pandas as pd
import numpy as np
import requests

from urllib.parse import urlencode
warnings.filterwarnings('ignore')


In [6]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://yadi.sk/d/guqki4UI4hFlXQ'

# Получаем ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Сохраняем
download_response = requests.get(download_url)
with open('data.zip', 'wb') as f: 
    f.write(download_response.content)

In [8]:
!unzip data.zip

!rm hidden_task.zip
!unzip open_task.zip

Archive:  data.zip
  inflating: hidden_task.zip         
replace open_task.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  open_task.zip
replace open_task/qid_query.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


# Ход действий:

## Решение Yandex Cup

In [11]:
qid_query = pd.read_csv("open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

# объединим таблицы
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

In [12]:
def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # max host rating
    top10 = max_by_host.sort_values(ascending=False)[:10] # top 10 url with gighest rating
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound

In [13]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # group by  qid and calc pfound
qid_max = qid_pfound.idxmax() #  qid with max pfound

qid_query[qid_query["qid"] == qid_max]

,qid,query
12,295761,гугл переводчик


In [14]:
qid_pfound.mean()

0.4603173929969002

## Свое решение

In [32]:
def calculate_pfound(df, pBreak=0.15):
    p_break = 0.15

    # удаляем дубликаты взаимодействий
    df = df.drop_duplicates(keep='first')

    # оставляем только максимально релевантный документ для каждого хоста
    df = df.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    
    # берем топ-10 hostid с максимальными рейтингами
    top_K = df.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(10)
    top_K['count'] = top_K.groupby('qid').cumcount()

    # для расчета pLook
    top_K['1-Rel'] = (1 - top_K['rating']).shift(1) 
    top_K['1-pBreak'] = 1 - pBreak 
    
    # for pLook[1] = 1
    top_K.loc[top_K['count'] == 0, ['1-Rel', '1-pBreak']] = 1

    # count поможет посчитать pLook
    top_K['count'] = top_K['1-Rel'] * top_K['1-pBreak']
    top_K['pLook'] = top_K.groupby('qid')['count'].cumprod()
    
    # pFound@K - матрично умножим
    top_K['pFound'] = top_K['pLook'] * top_K['rating']
    total_pfound = top_K.groupby('qid')['pFound'].sum()

    return total_pfound


In [36]:
print("Yandex PFound:")
print(qid_pfound, '\n')

print("Custom PFound:")
print(calculate_pfound(qid_url_rating_hostid))

Yandex PFound:
qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
dtype: float64 

Custom PFound:
qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
Name: pFound, dtype: float64


## Средний pFound:

In [38]:
print("Yandex mean PFound:")
print(qid_pfound.mean(), '\n')
print("Custom mean PFound:")
mean_pfound = calculate_pfound(qid_url_rating_hostid).mean()
print(mean_pfound)


Yandex mean PFound:
0.4603173929969002 

Custom mean PFound:
0.4603173929969002


Аутпут pFound c применением mean() соответствует qid_pfound.mean() Яндекса